In [5]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
from datetime import datetime, timedelta

import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

UndefVarError: UndefVarError: `from` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [ ]:
class CryptoEndpoint:
    listingsEndpoint = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest"
    latestQuotes = "https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/latest"
    historicalQuotes = "https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/historical"

    def __init__(self, apikey) -> None:
        self.headers = {
            'Accepts': 'application/json',
            'X-CMC_PRO_API_KEY': apikey,
        }
        self.coinsInfo = {}
        self.coinsIds = []

    def GetCoinIdentifiers(self):
        session = Session()
        session.headers.update(self.headers)
        response = session.get(
            url=self.listingsEndpoint,
            params={
                "limit": 10,
                "price_min": 1,
                "price_max" : 2
            }
        )

        data = json.loads(response.text)

        for coin in data.get("data", []):
            self.coinsInfo[coin["name"]] = coin["id"]
            self.coinsIds.append(coin["id"])

        print(f"Loaded {len(self.coinsInfo)} coins.")
        return (self.coinsInfo, self.coinsIds)

    def GetCoinLatestPrices(self, coin_id):

        session = Session()
        session.headers.update(self.headers)
        response = session.get(url=self.latestQuotes, params={"id": coin_id})
        data = json.loads(response.text)

        prices = {}
        for coin_id, info in data.get("data", {}).items():
            quote = info["quote"]["USD"]["price"]
            prices[coin_id] = quote

        return prices

    def GetSampleCoinHistoricalData(self, coin_id, days=4):
        session = Session()
        session.headers.update(self.headers)

        end_time = datetime.utcnow()
        start_time = end_time - timedelta(days=days)

        prices = {}

        params = {
            "id": coin_id,
            "time_start": start_time.isoformat(),
            "time_end": end_time.isoformat(),
            "interval": "24h",
        }

        response = session.get(url=self.historicalQuotes, params=params)
        data = json.loads(response.text)
        print(data["status"]["error_code"])
        coin_data = data.get("data", {})

        historicals = []

        for quote in coin_data["quotes"]:
            date = quote.get("timestamp") or quote.get("time_open")
            price = quote["quote"]["USD"]["price"]
            volume = quote["quote"]["USD"]["volume_24h"]
            historicals.append([price,volume])

        returnData = {coin_id: historicals}
        return returnData

In [ ]:
ce = CryptoEndpoint(input("Enter API-KEY"))
(coinsInfo, coinsIds) = ce.GetCoinIdentifiers()
data = []
for coinId in coinsIds:
  coinSample = ce.GetSampleCoinHistoricalData(coinId)
  if len(list(coinSample.values())[0]) < 1:
    continue
  data.append(coinSample)
print(coinsInfo)
print(data)

dataX = [list(coinData.values())[0] for coinData in data]
dataY = [list(coinData.keys())[0] for coinData in data]

In [1]:
#sample ltsm from geeks for geeks
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, h0=None, c0=None):
        if h0 is None or c0 is None:
            h0 = torch.zeros(self.layer_dim, x.size(
                0), self.hidden_dim).to(x.device)
            c0 = torch.zeros(self.layer_dim, x.size(
                0), self.hidden_dim).to(x.device)

        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])  # Take last time step
        return out

UndefVarError: UndefVarError: `class` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [2]:

trainX = torch.tensor(dataX, dtype=torch.float32)
trainY = torch.tensor([seq[-1][0] for seq in dataX], dtype=torch.float32)
trainY = trainY.view(-1, 1)
model = LSTMModel(input_dim=2, hidden_dim=100, layer_dim=1, output_dim=1)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
trainX

UndefVarError: UndefVarError: `torch` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [3]:
num_epochs = 100
h0, c0 = None, None

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    outputs = model(trainX, h0, c0)

    loss = criterion(outputs, trainY)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

UndefVarError: UndefVarError: `None` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [4]:
model.eval()

with torch.no_grad():
    predicted = model(trainX)  # shape: (num_coins, 1)
    actual_prices = [seq[-1][0] for seq in dataX]  # real final price per coin
    predicted_np = predicted.numpy().flatten()
    actual_np = np.array(actual_prices)
    plt.figure(figsize=(10, 5))
    plt.plot(actual_np, label="Actual Closing Price")
    plt.plot(predicted_np, label="Predicted Price", linestyle='--')
    plt.title("LSTM Coin Price Predictions")
    plt.xlabel("Coin Index")
    plt.ylabel("Price")
    plt.legend()
    plt.show()




UndefVarError: UndefVarError: `model` not defined in `Main`
Suggestion: check for spelling errors or missing imports.